# CSV to DXF converter

This script gets a file produced by the ski scanner, which is a CSV type containing information on displacement and width, and transforms it into a dxf file which is the input for the laser cutting machine used at Pomoca.

## CSV file read


[note] For now the file is a manual mock file with manual measurements

In [5]:
import pandas as pd

df = pd.read_csv("input.csv", sep = ";")

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   L       85 non-null     int64  
 1   W       85 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 1.5 KB
None
